In [ ]:
Banker Customer Recommendation Agentic AI

1. Introduction
1.1. Project Overview: Banker Customer Recommendation

This project demonstrates an Agentic AI framework designed to automate and enhance 
the process of generating personalized customer product recommendations within a banking context.

2. Core Functionality

The Agentic AI solution systematically executes the following sequence of operations:

- Recommendation Logic Definition: Explicitly defines the business rules and constraints 
  governing customer-product matching and recommendation generation.

- Customer Data Generation: Programmatically generates or simulates a CSV file containing 
  comprehensive customer profile and transaction data.

- Product Assignment and Augmentation: Applies the defined recommendation logic to 
  the customer data, systematically adding recommended banking products (e.g., checking account, savings accounts)
  to each customer record.


In [ ]:

3. Setup: Environment and Client Initialization

This phase focuses on establishing the necessary runtime environment and importing core dependencies required for the Agentic AI solution.

3.1. Dependency Import

We will begin by importing the following essential packages and modules:

- Standard Libraries: The built-in json library for data serialization and handling.

- Custom Logic: The core BankerCustomerRecommendation Python module, which contains the product recommendation logic.

- Data Module: The get_customers function, sourced from the Customer_data module, responsible for generating or fetching the customer dataset.

- Environment Management: The python-dotenv library to securely manage and load environment variables (e.g., API keys, configuration settings) from a .env file.


In [ ]:
import json
import csv
from ..src.models.recommend_products_v2 import Customer, ProductRecommender
from ..src.data.Customers_data import get_customers
from dotenv import load_dotenv
_ = load_dotenv()

VIP Member; Certificate of Deposit (Silver CD); Tech Professional Investment Plan; Retirement Growth Plan (RGP); Overdraft Protection Plan; Cash-Back Debit Rewards


In [ ]:
4. LLM Client Initialization and Configuration

This stage establishes the connection and configuration for interacting with the Large Language Model (LLM) service
We import the necessary Python modules:

- Standard Library: The os library for operating system interactions, primarily to manage 
  environment variables.

- AI Client: The aisuite library, imported as ai, which serves as the core client for interacting 
   with the chosen AI service (e.g., Google, OpenAI, etc.).

- Custom Models: The Customer and ProductRecommender classes are imported from the src.models.recommend_products_v2 
   module. These models define the data structure and business logic that the LLM will leverage

- The variable openai_api_key is assigned the value of the OPENAI_API_KEY environment variable using os.getenv(). 
  This best practice ensures that sensitive credentials are not hardcoded.

In [ ]:
import os
from openai import OpenAI
import aisuite as ai
from ..src.models.recommend_products_v2 import Customer, ProductRecommender

# Set your API key as an environment variable or pass enviroment varable your open api key, os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY'   
openai_api_key = os.getenv("OPENAI_API_KEY") 

client = ai.Client() 


In [ ]:
5. Business Logic Definition: Banker Recommendation Tool

This section details the definition of the core business logic function, 
which is designed to be exposed to the Large Language Model (LLM) as a callable tool. 

This function executes the customer-specific recommendation process.
The banker_recommendation_tool function is a specialized utility that accepts 
a customer's name and returns their profile details along with a personalized product recommendation.

In [3]:
def banker_recommendation_tool(name: str):
    customers = get_customers()
    recommender = ProductRecommender()
    
    for c in customers:
        if c["name"].lower() == name.lower():

            # Convert dict into Customer object
            customer_obj = Customer(
                name=c["name"],
                age=c.get("age"),
                address=c.get("address"),
                occupation=c.get("occupation"),
                balance=c.get("balance"),
                account_type=c.get("account_type")
            )
            return {
                "name": c["name"],
                "occupation": c["occupation"], 
                "balance": c["balance"],
                "recommendation": recommender.recommend(customer_obj) 
            }
    return {"error": f"No customer found with name {name}"}



In [ ]:
6.This TOOLS list defines the structure and metadata for the banker_recommendation_tool function,
 specifying its type, name, description, and expected parameters. This configuration enables
 seamless integration with the LLM, allowing it to invoke the function as needed during interactions.

In [4]:
import json

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "banker_recommendation_tool",
            "description": "Return the banking recommendation for a given customer name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"}
                },
                "required": ["name"]
            }
        }
    }
]


In [ ]:
7. LLM Interaction and Recommendation Generation

This section illustrates the interaction with the LLM to generate personalized banking recommendations.
It constructs a message prompt, invokes the LLM with the defined tools, and processes the response.
Message and prompt structure
prompt = "Generate a banking recommendation for Maria Lopez."

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Generate a banking recommendation for Maria Lopez." #John Smith
    }
]

response = client.chat.completions.create(
    model="openai:gpt-5.1",
    messages=messages,
    tools=TOOLS
)

msg = response.choices[0].message
print("MODEL MESSAGE:", msg)


In [ ]:
8. Tool Invocation and Execution

This code block demonstrates the crucial step of invoking the custom tool when 
the Large Language Model (LLM) determines that external information is required to fulfill the user's request.

Execution Flow

- Tool Call Detection: The if msg.tool_calls: condition checks the LLM's response (msg) for the presence of a tool call.

- Argument Extraction: If a tool call is found, the code extracts the first call (msg.tool_calls[0]) and parses the arguments 
    (e.g., the customer's name) provided by the LLM from a JSON string into a Python dictionary (args = json.loads(...)).

- Tool Execution: The extracted arguments are then passed to the custom function, banker_recommendation_tool(**args), 
  using argument unpacking. The result of the personalized recommendation logic is stored in tool_response.

- Output: The content of the tool_response (the structured customer data and product recommendation) is then 
  printed to the console for inspection.

This process enables the Agentic AI to dynamically call and execute the proprietary banking logic, treating it as an 
extension of the LLM's capabilities.

In [ ]:
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

if msg.tool_calls:
    tool_call = msg.tool_calls[0]
    args = json.loads(tool_call.function.arguments)

    tool_response = banker_recommendation_tool(**args)
    print("TOOL RESPONSE:", tool_response)


In [ ]:
9.Bulk Recommendation Generation and CSV Export

This script block executes the core batch processing of customer data to generate and persist product recommendations for the entire customer base in a structured format.
    
Execution Flow

- Dependency Import: Necessary classes (Customer, ProductRecommender) and the data source function (get_customers) are imported. 
Note: Although ProductRecommender is imported, the function used is recommend_products.

- File Initialization: An output file named customer_recommendations.csv is opened in write mode ("w"), and a CSV writer is initialized.

- Data Loading & Header: The complete list of customers is retrieved via get_customers(), and the CSV file is immediately written with 
a header row detailing all customer attributes and the Recommendations column.

- Iterative Processing: The code loops through every customer record (c):
The dictionary record is transformed into a structured Customer object. The recommend_products() function (presumably an alias or wrapper 
for the logic in ProductRecommender) is called to generate the personalized recommendation.

        A new row containing all customer details and the generated recommendation is written to the CSV file.

In [ ]:
import csv
from ..src.models.recommend_products_v2 import Customer, ProductRecommender
from ..src.data.Customers_data import get_customers

output_file = "customer_recommendations.csv"
customers = get_customers()
recommender = ProductRecommender()
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    writer.writerow([
        "Name", "Age", "Address", "Occupation", 
        "Balance", "Account Type", "Recommendations"
    ])
    
    for c in customers:
        cust_obj = Customer(
            name=c["name"],
            age=c["age"],
            address=c["address"],
            occupation=c["occupation"],
            balance=c["balance"],
            account_type=c["account_type"]
        )
        recs = recommender.recommend(cust_obj)

        # Write row
        writer.writerow([
            cust_obj.name,
            cust_obj.age,
            cust_obj.address,
            cust_obj.occupation,
            cust_obj.balance,
            cust_obj.account_type,
            recs
         ])

    print(f"CSV file '{output_file}' created successfully!")


VIP Member; Certificate of Deposit (Silver CD); Tech Professional Investment Plan; Retirement Growth Plan (RGP); Overdraft Protection Plan; Cash-Back Debit Rewards
CSV file 'customer_recommendations.csv' created successfully!
